<a href="https://github.com/timeseriesAI/tsai-rs" target="_parent"><img src="https://img.shields.io/badge/tsai--rs-Time%20Series%20AI%20in%20Rust-blue" alt="tsai-rs"/></a>

# Hyperparameter Optimization with Optuna

This notebook demonstrates how to optimize hyperparameters for time series models using **tsai-rs** with Optuna.

## Purpose

Hyperparameter optimization helps you find the best configuration for your model.

Optuna provides:
1. **Efficient search**: Bayesian optimization finds good params faster than grid search
2. **Pruning**: Early stopping of unpromising trials
3. **Visualization**: Understand parameter importance
4. **Parallelization**: Run multiple trials concurrently

## Install Dependencies

```bash
pip install optuna
cd crates/tsai_python
maturin develop --release
```

## Import Libraries

In [ ]:
import tsai_rs
import numpy as np
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances

print(f"tsai-rs version: {tsai_rs.version()}")
print(f"optuna version: {optuna.__version__}")
tsai_rs.my_setup()

## Load Data

In [ ]:
dsid = 'NATOPS'
X_train, y_train, X_test, y_test = tsai_rs.get_UCR_data(dsid, return_split=True)

n_vars = X_train.shape[1]
seq_len = X_train.shape[2]
n_classes = len(np.unique(y_train))

print(f"Dataset: {dsid}")
print(f"X_train shape: {X_train.shape}")
print(f"Variables: {n_vars}, Sequence length: {seq_len}, Classes: {n_classes}")

In [ ]:
# Standardize data
X_train_std = tsai_rs.ts_standardize(X_train.astype(np.float32), by_sample=True)
X_test_std = tsai_rs.ts_standardize(X_test.astype(np.float32), by_sample=True)

## Establish Baseline

In [ ]:
# Baseline configuration with default parameters
baseline_config = tsai_rs.InceptionTimePlusConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes
)

baseline_learner = tsai_rs.LearnerConfig(
    lr=1e-3,
    weight_decay=0.01,
    grad_clip=1.0
)

print(f"Baseline model config: {baseline_config}")
print(f"Baseline learner config: {baseline_learner}")

## Define Objective Function

The objective function defines what Optuna should optimize.

In [ ]:
def simulate_training(config_params):
    """Simulate training with given hyperparameters.
    
    In a real scenario, this would train the model and return validation accuracy.
    """
    # Simulate that certain parameter combinations work better
    lr = config_params['lr']
    weight_decay = config_params['weight_decay']
    depth = config_params.get('depth', 6)
    nf = config_params.get('nf', 32)
    
    # Simulate accuracy based on hyperparameters
    # (In reality, you would train and evaluate)
    base_acc = 0.7
    
    # Learning rate effect (optimal around 1e-3)
    lr_effect = -np.abs(np.log10(lr) + 3) * 0.05
    
    # Weight decay effect (optimal around 0.01)
    wd_effect = -np.abs(np.log10(weight_decay + 1e-10) + 2) * 0.02
    
    # Depth effect (optimal around 6)
    depth_effect = -np.abs(depth - 6) * 0.02
    
    # Number of filters effect (more is generally better up to a point)
    nf_effect = 0.1 * np.log(nf / 32)
    
    # Add some noise
    noise = np.random.normal(0, 0.02)
    
    accuracy = base_acc + lr_effect + wd_effect + depth_effect + nf_effect + noise
    accuracy = np.clip(accuracy, 0, 1)
    
    return accuracy

In [ ]:
def objective(trial: optuna.Trial):
    """Optuna objective function for hyperparameter optimization."""
    
    # Define search space
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-1, log=True)
    nf = trial.suggest_categorical('nf', [16, 32, 64, 96])
    depth = trial.suggest_int('depth', 3, 9, step=3)
    dropout = trial.suggest_float('dropout', 0.0, 0.5, step=0.1)
    
    # Create configuration
    config_params = {
        'lr': lr,
        'weight_decay': weight_decay,
        'nf': nf,
        'depth': depth,
        'dropout': dropout,
    }
    
    # Simulate training and get accuracy
    accuracy = simulate_training(config_params)
    
    return accuracy

## Run Optuna Study

In [ ]:
# Create study
study = optuna.create_study(
    direction='maximize',  # Maximize accuracy
    study_name='tsai_rs_hpo'
)

# Run optimization
n_trials = 50
print(f"Running {n_trials} trials...")
study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

## Analyze Results

In [ ]:
# Best trial
print("Best trial:")
print(f"  Value (accuracy): {study.best_value:.4f}")
print(f"  Parameters:")
for key, value in study.best_params.items():
    print(f"    {key}: {value}")

In [ ]:
# Study statistics
print("\nStudy statistics:")
print(f"  Number of finished trials: {len(study.trials)}")
print(f"  Best trial number: {study.best_trial.number}")

# Get trial history
trial_values = [t.value for t in study.trials if t.value is not None]
print(f"  Mean accuracy: {np.mean(trial_values):.4f}")
print(f"  Std accuracy: {np.std(trial_values):.4f}")
print(f"  Min accuracy: {np.min(trial_values):.4f}")
print(f"  Max accuracy: {np.max(trial_values):.4f}")

## Visualize Optimization

In [ ]:
import matplotlib.pyplot as plt

# Plot optimization history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Trial values over time
ax1 = axes[0]
trial_numbers = range(len(trial_values))
ax1.scatter(trial_numbers, trial_values, alpha=0.6, label='Trial value')

# Running best
running_best = [max(trial_values[:i+1]) for i in range(len(trial_values))]
ax1.plot(trial_numbers, running_best, 'r-', linewidth=2, label='Best so far')

ax1.set_xlabel('Trial')
ax1.set_ylabel('Accuracy')
ax1.set_title('Optimization History')
ax1.legend()

# Parameter distributions for top trials
ax2 = axes[1]
top_n = 10
sorted_trials = sorted(study.trials, key=lambda t: t.value if t.value else 0, reverse=True)[:top_n]
top_lrs = [t.params['lr'] for t in sorted_trials]
top_values = [t.value for t in sorted_trials]

ax2.scatter(top_lrs, top_values, s=100, c=range(top_n), cmap='viridis')
ax2.set_xscale('log')
ax2.set_xlabel('Learning Rate')
ax2.set_ylabel('Accuracy')
ax2.set_title(f'Top {top_n} Trials by Learning Rate')

plt.tight_layout()
plt.show()

## Parameter Importance

In [ ]:
# Calculate parameter importance
try:
    importances = optuna.importance.get_param_importances(study)
    
    print("Parameter importances:")
    print("-" * 40)
    for param, importance in importances.items():
        print(f"  {param}: {importance:.4f}")
        
    # Visualize
    fig, ax = plt.subplots(figsize=(10, 5))
    params = list(importances.keys())
    values = list(importances.values())
    
    ax.barh(params, values, color='steelblue', edgecolor='black')
    ax.set_xlabel('Importance')
    ax.set_title('Hyperparameter Importance')
    
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Could not compute importances: {e}")

## Create Optimized Configuration

In [ ]:
# Create tsai-rs configuration with best parameters
best_params = study.best_params

optimized_model_config = tsai_rs.InceptionTimePlusConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes,
    # Additional params would go here based on what the config supports
)

optimized_learner_config = tsai_rs.LearnerConfig(
    lr=best_params['lr'],
    weight_decay=best_params['weight_decay'],
    grad_clip=1.0
)

print("Optimized configuration:")
print(f"  Model: {optimized_model_config}")
print(f"  Learner: {optimized_learner_config}")

## Multi-Objective Optimization

Sometimes you want to optimize multiple objectives (e.g., accuracy and speed).

In [ ]:
def multi_objective(trial: optuna.Trial):
    """Multi-objective optimization: accuracy and inference speed."""
    
    # Search space
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    nf = trial.suggest_categorical('nf', [16, 32, 64, 96])
    depth = trial.suggest_int('depth', 3, 9, step=3)
    
    # Simulate accuracy
    accuracy = simulate_training({'lr': lr, 'weight_decay': 0.01, 'nf': nf, 'depth': depth})
    
    # Simulate inference speed (inversely related to model size)
    # Larger models are slower
    model_size = nf * depth
    inference_speed = 1.0 / (1 + model_size / 100)  # Normalized speed
    
    return accuracy, inference_speed

# Create multi-objective study
multi_study = optuna.create_study(
    directions=['maximize', 'maximize'],  # Maximize both
    study_name='tsai_rs_multi_objective'
)

print("Running multi-objective optimization...")
multi_study.optimize(multi_objective, n_trials=30, show_progress_bar=True)

In [ ]:
# Pareto front
print("\nPareto optimal trials:")
for trial in multi_study.best_trials:
    print(f"  Trial {trial.number}:")
    print(f"    Accuracy: {trial.values[0]:.4f}")
    print(f"    Speed: {trial.values[1]:.4f}")
    print(f"    Params: {trial.params}")

## Architecture Search

In [ ]:
def architecture_objective(trial: optuna.Trial):
    """Search across different architectures."""
    
    # Architecture selection
    arch = trial.suggest_categorical('architecture', [
        'InceptionTimePlus', 'ResNetPlus', 'TST', 'MiniRocket'
    ])
    
    # Common hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    
    # Simulate different base accuracies for different architectures
    base_accuracies = {
        'InceptionTimePlus': 0.85,
        'ResNetPlus': 0.82,
        'TST': 0.88,
        'MiniRocket': 0.84,
    }
    
    base_acc = base_accuracies[arch]
    lr_effect = -np.abs(np.log10(lr) + 3) * 0.05
    noise = np.random.normal(0, 0.02)
    
    accuracy = base_acc + lr_effect + noise
    return np.clip(accuracy, 0, 1)

# Run architecture search
arch_study = optuna.create_study(direction='maximize', study_name='arch_search')
arch_study.optimize(architecture_objective, n_trials=40, show_progress_bar=True)

print(f"\nBest architecture: {arch_study.best_params['architecture']}")
print(f"Best accuracy: {arch_study.best_value:.4f}")

## Summary

This notebook demonstrated hyperparameter optimization with Optuna:

### Key Concepts
1. **Define search space**: Use `trial.suggest_*` methods
2. **Objective function**: Returns the metric to optimize
3. **Study**: Manages the optimization process
4. **Analysis**: Understand parameter importance

### Optuna Features Used
- `suggest_float`: Continuous parameters (log scale optional)
- `suggest_categorical`: Discrete choices
- `suggest_int`: Integer parameters
- Multi-objective optimization
- Parameter importance analysis

### tsai-rs Integration
```python
# Use best params in tsai-rs config
config = tsai_rs.InceptionTimePlusConfig(
    n_vars=n_vars,
    seq_len=seq_len,
    n_classes=n_classes
)

learner_config = tsai_rs.LearnerConfig(
    lr=study.best_params['lr'],
    weight_decay=study.best_params['weight_decay']
)
```

In [ ]:
# Quick reference
print("Optuna + tsai-rs Quick Reference")
print("=" * 50)
print("\n# Define objective")
print("def objective(trial):")
print("    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)")
print("    return accuracy")
print("\n# Create and run study")
print("study = optuna.create_study(direction='maximize')")
print("study.optimize(objective, n_trials=100)")
print("\n# Get best parameters")
print("best_params = study.best_params")
print("best_value = study.best_value")